In [ ]:
# generate a random pair q0,v0 (probably sliding on a surface)
# solve for x_ipopt using ipopt using explicit contact forces
# we want loss = (x_lag - x_ipopt)^2 == 0
# compute the gradient of the loss with respect to the parameters
# update the parameters using SGD
# repeat

In [2]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

In [3]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
env = parse_contacts(mechanism, urdf, .5);
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);
xnext = MechanismState(mechanism);

In [ ]:
num_steps_default = 10
α_vect_default = [1.^i for i in 1:num_steps_default]
c_vect_default = [200.+min.(2.^i,200) for i in 1:num_steps_default]
I_vect_default = 1e-16*ones(num_steps_default)
z = vcat(α_vect_default,c_vect_default,I_vect_default)

num_iter = 1
batch_size = 1
α_sgd = 0.1

xmax = 2.
zmax = 1.
vmax = 1.

for iter = 1:num_iter
    loss_batch = 0.
    g_batch = zeros(length(z))
    
    for j = 1:batch_size
        # half in contact, half not
        if j%2==0 
            q0 = vcat([1., 0., 0., 0.], (rand(2) .- .5) .* 2. .* xmax, 0.)
        else
            q0 = vcat([1., 0., 0., 0.], (rand(2) .- .5) .* 2. .* xmax, (rand(1) .- .5) .* 2. .* zmax)
        end
        v0 = vcat(zeros(3),(rand(3) .- .5) .* 2. .* vmax)
        u0 = zeros(sim_data.num_v)
        z0 = zeros(sim_data.num_contacts*(sim_data.β_dim+2))

        set_configuration!(x0, q0)
        set_velocity!(x0, v0)
        setdirty!(x0)
        traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,implicit_contact=false)
        qnext = traj[1:sim_data.num_q,end]
        vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,end]
        
        τ_ip, x_sol_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=true);
        τ_auglag, x_sol_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=false,α_vect=z[1:num_steps_default],c_vect=z[num_steps_default+1:2*num_steps_default],I_vect=z[2*num_steps_default+1:3*num_steps_default])
        J = ForwardDiff.jacobian(z̃ -> solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=false,α_vect=z̃[1:num_steps_default],c_vect=z̃[num_steps_default+1:2*num_steps_default],I_vect=z̃[2*num_steps_default+1:3*num_steps_default])[2], z)
    
        loss = .5 * dot(x_sol_ip - x_sol_auglag, x_sol_ip - x_sol_auglag)
        
        loss_batch += loss
        g_batch += J'*(x_sol_ip - x_sol_auglag)
    end
    loss_batch /= batch_size
    g_batch /= batch_size
    
    z = z + α_sgd .* g_batch
    z = min.(200.,max.(1e-6, z))
    
    println(loss_batch)
    println(z[1:num_steps_default])
    println(z[num_steps_default+1:2*num_steps_default])
    println("***")
end